# ITU 5G Energy Consumption Challenge - GitData

<font color="red">Please make a copy of this Jupyter Notebook to your own Google Drive or local machine before changing anything.</font>

This Jupyter Notebook will also be updated regularly on GitHub: https://github.com/GitData-GA/itu5g/tree/main/code/ITU_5G_Energy_Consumption_Challenge.ipynb

In [1]:
# Import all libraries here
import random
random.seed(88)
import pandas as pd
import numpy as np
np.random.seed(88)
import tensorflow as tf
tf.random.set_seed(88)
from tensorflow import keras
keras.utils.set_random_seed(88)
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import mean_absolute_percentage_error
from matplotlib import pyplot as plt
from keras import backend as K

In [2]:
# Output format
# Description: The file contains days, hours, and base stations for which we
# need energy consumption estimation and will be used for scoring predictions.
power_pred = pd.read_csv('https://data.gd.edu.kg/bwz7t527/power_consumption_prediction.csv')
# Dataset 1
# Description: Energy consumption data (ECdata.csv): include hour-level energy
# consumption specifications (e.g., total energy consumption of the base stations).
ECdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/ECdata.csv')
# Dataset 2
# Description: Cell-level data (CLdata.csv): include hour-level counters,
# including service compliance counters (e.g., load) and energy-saving methods
# counters (e.g., duration of energy saving mode activation).
CLdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/CLdata.csv')
# Dataset 3
# Description: Base Station basic information (BSinfo.csv): include
# configuration parameters and hardware attributes
BSinfo = pd.read_csv('https://data.gd.edu.kg/bwz7t527/BSinfo.csv')

In [3]:
data = ECdata.merge(CLdata, on=["Time", "BS"], how="inner")
data = data.merge(BSinfo, on="BS", how="inner")
data = data.sort_values(by='Time', ascending=True)
# Convert date strings to datetime objects
data['Time'] = pd.to_datetime(data['Time'])

# Sort the dataset by date
data.sort_values(by='Time', inplace=True)

# Split Year, Month, Day, and Hour
data['Year'] = data['Time'].dt.year
data['Month'] = data['Time'].dt.month
data['Day'] = data['Time'].dt.day
data['Hour'] = data['Time'].dt.hour

# Extract target variable 'Energy'
y = data['Energy']

# Drop unnecessary columns (e.g., 'Time', 'Energy')
data.drop(columns=['Time', 'Energy'], inplace=True)

# One-hot encode categorical features
categorical_cols = ['BS', 'CellName_x', 'CellName_y', 'RUType', 'Mode', ]
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Standardize numerical features
scaler = StandardScaler()
data_encoded[data_encoded.columns] = scaler.fit_transform(data_encoded[data_encoded.columns])

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data_encoded, y, test_size=0.2, shuffle=False)

# Convert data to numpy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [14]:
X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
lstm_model = keras.Sequential([
    layers.LSTM(units=64, recurrent_dropout=0.25),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu', kernel_regularizer=l1(0.001)),
    layers.Dense(units=1)
])

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_absolute_percentage_error')

# Train the model
history = lstm_model.fit(X_train_lstm, y_train, epochs=14, batch_size=256, validation_split=0.2)

# Fit model on validation set and get error
y_pred = lstm_model.predict(X_test_lstm)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {mape:.5f}%')

Epoch 1/14
315/315 [==============================] - 12s 21ms/step - loss: 41.5844 - val_loss: 11.2261
Epoch 2/14
315/315 [==============================] - 7s 23ms/step - loss: 13.2039 - val_loss: 8.0117
Epoch 3/14
315/315 [==============================] - 6s 20ms/step - loss: 11.4420 - val_loss: 7.2548
Epoch 4/14
315/315 [==============================] - 7s 23ms/step - loss: 10.5712 - val_loss: 6.7338
Epoch 5/14
315/315 [==============================] - 6s 19ms/step - loss: 9.8297 - val_loss: 6.4149
Epoch 6/14
315/315 [==============================] - 7s 23ms/step - loss: 9.2743 - val_loss: 6.3521
Epoch 7/14
315/315 [==============================] - 6s 20ms/step - loss: 8.9089 - val_loss: 6.1617
Epoch 8/14
315/315 [==============================] - 7s 24ms/step - loss: 8.6515 - val_loss: 6.1134
Epoch 9/14
315/315 [==============================] - 7s 22ms/step - loss: 8.3690 - val_loss: 6.0582
Epoch 10/14
315/315 [==============================] - 6s 20ms/step - loss: 8.1222 - 

In [15]:
model = keras.Sequential([
    layers.Dense(units=64, activation='relu', kernel_regularizer=l1(0.001)),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu', kernel_regularizer=l1(0.001)),
    layers.Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_percentage_error')

# Train the model
history = model.fit(X_train, y_train, epochs=14, batch_size=256, validation_split=0.2)

# Fit model on validation set and get error
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {mape:.5f}%')

Epoch 1/14
315/315 [==============================] - 3s 8ms/step - loss: 36.1639 - val_loss: 12.5693
Epoch 2/14
315/315 [==============================] - 2s 7ms/step - loss: 15.8767 - val_loss: 10.9800
Epoch 3/14
315/315 [==============================] - 2s 7ms/step - loss: 14.6905 - val_loss: 10.6781
Epoch 4/14
315/315 [==============================] - 2s 7ms/step - loss: 13.8490 - val_loss: 9.7771
Epoch 5/14
315/315 [==============================] - 4s 11ms/step - loss: 12.8657 - val_loss: 9.3302
Epoch 6/14
315/315 [==============================] - 3s 8ms/step - loss: 11.9433 - val_loss: 8.9967
Epoch 7/14
315/315 [==============================] - 2s 7ms/step - loss: 11.4745 - val_loss: 8.8121
Epoch 8/14
315/315 [==============================] - 2s 7ms/step - loss: 11.0221 - val_loss: 8.5965
Epoch 9/14
315/315 [==============================] - 2s 7ms/step - loss: 10.6493 - val_loss: 8.4694
Epoch 10/14
315/315 [==============================] - 3s 10ms/step - loss: 10.4402 - v